In [1]:
# PySpark
from pyspark.sql import SparkSession
import os

os.environ["PYSPARK_PYTHON"]="/home/pc/g5_env/bin/python39"

In [2]:
# Schema declaration
from pyspark.sql.types import StructType,StructField, StringType, IntegerType
#map reduce i think
from functools import reduce
#pyspark sql functions
from pyspark.sql import functions as f
# storage level
from pyspark import StorageLevel

In [3]:
spark = SparkSession.builder.master("local[12]")\
            .appName("Read CSV")\
            .config('spark.executor.memory', '10g')\
            .config('spark.driver.maxResultSize', '5g')\
            .config('spark.driver.memory', '10g')\
            .getOrCreate()
sparkContext = spark.sparkContext
spark

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/05 05:10:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/10/05 05:10:38 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
spark.conf.get('spark.sql.files.maxPartitionBytes')

'134217728b'

In [5]:
spark.conf.set('spark.sql.files.maxPartitionBytes', '256000000b')
spark.conf.get('spark.sql.files.maxPartitionBytes')

'256000000b'

In [6]:
df = spark.read.option("wholetext",False)\
    .text("/home/pc/ozj/wiki-data-zh-csv-test-remove-html.txt")

In [7]:
df.show(40,truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [8]:
df = df.dropna(how='any').where(df.value != ' ')

In [9]:
df.rdd.getNumPartitions()

12

In [10]:
df.rdd.getNumPartitions()

12

In [11]:
df = df.repartition(48)

In [12]:
df.rdd.getNumPartitions()

48

### <font color='red'>Sentence Tokenize</font>

In [13]:
#test harvest text

from harvesttext import HarvestText
ht = HarvestText()

para = """吴国雄（），中国大气动力学和气候动力学家。 1943年出生于广东潮阳。1966年毕业于南京气象学院。1983年获英国伦敦大学理学博士学位。1997年当选为中国科学院院士。中国科学院大气物理研究所研究员。曾任大气科学和地球流体力学数值模拟国家重点实验室主任、国际气象和大气科学协会中国国家委员会主席。"""

print(ht.cut_sentences(para))

['吴国雄（），中国大气动力学和气候动力学家。', '1943年出生于广东潮阳。', '1966年毕业于南京气象学院。', '1983年获英国伦敦大学理学博士学位。', '1997年当选为中国科学院院士。', '中国科学院大气物理研究所研究员。', '曾任大气科学和地球流体力学数值模拟国家重点实验室主任、国际气象和大气科学协会中国国家委员会主席。']


In [14]:
from harvesttext import HarvestText

def sentence_tokenize_zh(index, text):
    ht = HarvestText()
    return index, ht.cut_sentences(text)

def f(x): return x

In [15]:
# Adding indexes to all sentence
# Swapping the column
# Splitting the sentence
#RDD1 = 
RDD1 = df.rdd.map(lambda x: x[0]).zipWithIndex()\
.map(lambda x: sentence_tokenize_zh(x[1], x[0]))\
.filter(lambda x : x[1] != '')\
.flatMapValues(f)\
.persist(StorageLevel.MEMORY_ONLY)
# .toDF().show(100, truncate = False)

In [16]:
RDD1.map(lambda x: (x[1],)).toDF().show(20,truncate=False)

22/10/05 05:12:05 WARN BlockManager: Task 61 already completed, not releasing lock for rdd_19_0


22/10/05 05:12:09 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 7 (TID 62): Attempting to kill Python Worker
+------------------------------------------------------------------------------------------------+
|_1                                                                                              |
+------------------------------------------------------------------------------------------------+
|吴国雄（），中国大气动力学和气候动力学家。                                                      |
|1943年出生于广东潮阳。                                                                          |
|1966年毕业于南京气象学院。                                                                      |
|1983年获英国伦敦大学理学博士学位。                                                              |
|1997年当选为中国科学院院士。                                                                    |
|中国科学院大气物理研究所研究员。                                                                |
|曾任大气科学和地球流体力学数值模拟国家重点实验室主任、国际气象和大气科学协会中国国家委员会主席。|
|神經母細胞瘤

Remove html code and header (== like this ==)

## <font color='red'>Text Cleaning</font>


#### Remove HTML tag

In [17]:
def remove_html_tags(text):
    #Remove html tags from a string
    import re
    clean = re.compile('<.*?>')
    return re.sub(clean, ' ', text)

#### Remove Emoticons

In [18]:
from emot.emo_unicode import EMOTICONS_EMO

def remove_emoticons(text):
    for emot in EMOTICONS_EMO:
        text = text.replace(emot, '')
    return text

#RDD2 = RDD.map(lambda x: remove_emoticons(x)).persist(StorageLevel.MEMORY_ONLY)

In [19]:
RDD2 = RDD1.map(lambda x : remove_html_tags(x[1]))\
.map(lambda x: remove_emoticons(x))\
.persist(StorageLevel.MEMORY_ONLY)
# .map(lambda x : sent_tokenize(x))\


In [20]:
RDD3 = RDD2.zipWithIndex()\
.map(lambda x: sentence_tokenize_zh(x[1], x[0]))\
.flatMapValues(f)\
.persist(StorageLevel.MEMORY_ONLY)

In [21]:
RDD3.map(lambda x: (x[1],)).toDF().show(20,truncate=False)

22/10/05 05:20:55 WARN BlockManager: Task 111 already completed, not releasing lock for rdd_29_0


22/10/05 05:20:59 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 13 (TID 112): Attempting to kill Python Worker
+------------------------------------------------------------------------------------------------+
|_1                                                                                              |
+------------------------------------------------------------------------------------------------+
|吴国雄（），中国大气动力学和气候动力学家。                                                      |
|1943年出生于广东潮阳。                                                                          |
|1966年毕业于南京气象学院。                                                                      |
|1983年获英国伦敦大学理学博士学位。                                                              |
|1997年当选为中国科学院院士。                                                                    |
|中国科学院大气物理研究所研究员。                                                                |
|曾任大气科学和地球流体力学数值模拟国家重点实验室主任、国际气象和大气科学协会中国国家委员会主席。|
|神經母細

#### 1) Remove Emoji, URL, Phone Number, Currency, Punctuations, Digit, E-mail
#### 2) To Lower

In [23]:
from cleantext import clean

def clean_text(text): 
    return str(clean(text,
            fix_unicode=True,               # fix various unicode errors
            to_ascii=False,                  # transliterate to closest ASCII representation
            lower=True,                     # lowercase text
            no_line_breaks=True,           # fully strip line breaks as opposed to only normalizing them
            no_urls=True,                  # replace all URLs with a special token
            no_emails=True,                # replace all email addresses with a special token
            no_phone_numbers=True,         # replace all phone numbers with a special token
            no_numbers=True,               # replace all numbers with a special token
            no_digits=False,                # replace all digits with a special token
            no_currency_symbols=True,      # replace all currency symbols with a special token
            no_punct=True,                 # remove punctuations
            no_emoji=True,
            replace_with_punct="",          # instead of removing punctuations you may replace them
            replace_with_url="",
            replace_with_email="",
            replace_with_phone_number="",
            replace_with_number="",
            replace_with_digit="",
            replace_with_currency_symbol="",
            lang="en"                       # set to 'de' for German special handling
            ))

In [24]:
RDD4 = RDD3.map(lambda x: (x[1],))\
.map(lambda x : (clean_text(x)))\
.filter(lambda x : x != '')\
.persist(StorageLevel.MEMORY_ONLY)

In [26]:
RDD4.map(lambda x: (x, ))\
.filter(lambda x : x != '')\
.persist(StorageLevel.MEMORY_ONLY)\
.toDF().show(20,truncate=False)

22/10/05 05:26:55 WARN BlockManager: Task 115 already completed, not releasing lock for rdd_46_0


22/10/05 05:26:59 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 21 (TID 116): Attempting to kill Python Worker
+--------------------------------------------------------------------------------------------+
|_1                                                                                          |
+--------------------------------------------------------------------------------------------+
|吴国雄中国大气动力学和气候动力学家                                                          |
|1943年出生于广东潮阳                                                                        |
|1966年毕业于南京气象学院                                                                    |
|1983年获英国伦敦大学理学博士学位                                                            |
|1997年当选为中国科学院院士                                                                  |
|中国科学院大气物理研究所研究员                                                              |
|曾任大气科学和地球流体力学数值模拟国家重点实验室主任国际气象和大气科学协会中国国家委员会主席|
|神經母細胞瘤neuroblastoma是一種在特定神經組織中形成的

In [36]:
import re

RDD5 = RDD4.map(lambda x: (re.sub(r'[^\u2E80-\u2FD5\u3190-\u319f\u3400-\u4DBF\u4E00-\u9FCC\uF900-\uFAAD ]', '', x), ))\
    .persist(StorageLevel.MEMORY_ONLY)

In [38]:
RDD5.map(lambda x: (x[0],)).toDF().show(truncate=False)

22/10/05 05:55:30 WARN BlockManager: Task 302 already completed, not releasing lock for rdd_72_0


22/10/05 05:55:34 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 40 (TID 303): Attempting to kill Python Worker
+--------------------------------------------------------------------------------------------+
|_1                                                                                          |
+--------------------------------------------------------------------------------------------+
|吴国雄中国大气动力学和气候动力学家                                                          |
|年出生于广东潮阳                                                                            |
|年毕业于南京气象学院                                                                        |
|年获英国伦敦大学理学博士学位                                                                |
|年当选为中国科学院院士                                                                      |
|中国科学院大气物理研究所研究员                                                              |
|曾任大气科学和地球流体力学数值模拟国家重点实验室主任国际气象和大气科学协会中国国家委员会主席|
|神經母細胞瘤是一種在特定神經組織中形成的癌症           

In [71]:
final_df = RDD5.map(lambda x: (x[0],))\
.toDF()

22/10/05 06:04:04 WARN BlockManager: Task 326 already completed, not releasing lock for rdd_72_0


In [72]:
final_df.show(100,truncate=False)

22/10/05 06:04:09 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 88 (TID 327): Attempting to kill Python Worker
+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|_1                                                                                                                                                                                                                                                                                                                                                 |
+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [73]:
final_df = final_df.coalesce(1)

In [74]:
final_df.write.parquet("hdfs://g5.bigtop.it:8020/user/root/wikidata_zh_preprocessed.parquet")

In [ ]:
# re-save again the dataframe
final_df.replace.parquet("hdfs://g5.bigtop.it:8020/user/root/wikidata_zh_preprocessed.parquet")

In [75]:
test_df = spark.read.parquet("hdfs://g5.bigtop.it:8020/user/root/wikidata_zh_preprocessed.parquet")

In [76]:
test_df.show(10,truncate=False)

+--------------------------------------------------------------------------------------------+
|_1                                                                                          |
+--------------------------------------------------------------------------------------------+
|吴国雄中国大气动力学和气候动力学家                                                          |
|年出生于广东潮阳                                                                            |
|年毕业于南京气象学院                                                                        |
|年获英国伦敦大学理学博士学位                                                                |
|年当选为中国科学院院士                                                                      |
|中国科学院大气物理研究所研究员                                                              |
|曾任大气科学和地球流体力学数值模拟国家重点实验室主任国际气象和大气科学协会中国国家委员会主席|
|神經母細胞瘤是一種在特定神經組織中形成的癌症                                                |
|它最常從一側腎上腺開始但也可以在頸部胸部腹部或脊髓中發展                                    |
|症狀包括腹部頸部或胸部腫塊或皮膚下無痛的藍色腫塊      